In [110]:
## make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

%load_ext autoreload
%autoreload

import pandas as pd
from collections import namedtuple

from pa_lib.file import (
    project_dir,
    load_bin,
    load_xlsx,
    store_bin,
)
from pa_lib.data import as_dtype, dtFactor
from pa_lib.util import cap_words, collect, value

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data sets

In [111]:
with project_dir("axinova"):
    ax_data = load_bin("ax_data.feather")
    spr_data = load_bin("spr_data.feather")

17:03:09 [INFO] Started loading binary file ...
17:03:09 [INFO] Reading from file C:\Users\kpf\data\axinova\ax_data.feather
17:03:09 [INFO] ... finished loading binary file in 0.27s (0.94s CPU)
17:03:09 [INFO] Started loading binary file ...
17:03:09 [INFO] Reading from file C:\Users\kpf\data\axinova\spr_data.feather
17:03:09 [INFO] ... finished loading binary file in 0.01s (0.02s CPU)


# Filter data for merging

In [112]:
spr_data.dropna(subset=["PF"], inplace=True)

# Find mapping of Flächen to Stations

In [113]:
pd.DataFrame(dict(Station=ax_data.Station.unique()))

,Station
0,Aarau
1,Basel SBB
2,Bellinzona
3,Bern
4,Biel/Bienne
5,Brig
6,Chur
7,Fribourg
8,Genève Aéroport
9,Genève Cornavin


In [ ]:
spr_data.columns

## Problem cantons: ZH, GE, VD, BE, TI (> 1 station each)

In [114]:
def print_list(column, condition):
    print(collect(spr_data.loc[condition, column], sep="\n"))

In [115]:
print_list("Adresszusatz", condition=(spr_data["Gebiet Code"] == "ZH"))

Gleis 2, Sektor C, Fahrtreppenaufgang
Gleis 3, Sektor C, Fahrtreppenaufgang
Gleis 31/32, Sektor B, Aufgang Halle Löwenstrasse
Gleis 31/32, Sektor B, Wartezone
Gleis 33/34, Sektor B, Aufgang Halle Löwenstrasse
Gleis 33/34, Sektor B, Wartezone
Gleis 4, Sektor A, vis-à-vis Hausperron
Gleis 4, Sektor B, vis-à-vis Hausperron
Gleis 4, Sektor C, vis-à-vis Hausperron
Gleis 41, Linien 5,6,7,9,12, Sektor B
Gleis 41, Linien 5,6,7,9,12, Sektor C
Gleis 44, Linien 5,6,7,9,12, Sektor B
Gleis 44, Linien 5,6,7,9,12, Sektor C
Halle Kopfperron
Halle Landesmuseum, Abgänge S-Bahn Perron Gleis 41+42 / 43+44
Halle Löwenstrasse, Aufgang Gleis 6
Halle Löwenstrasse, Aufgang Gleis 7
Halle Löwenstrasse, Aufgang Shopville
Halle Museumstrasse, Passage Bahnhofstrasse
Halle Museumstrasse, Passage Löwenstrasse
Haupzugang, Kiosk
Passage Löwenstrasse
Passage Sihlquai, Liftschacht Gleis 10
Passage Sihlquai, Liftschacht Gleis 11
Passage Sihlquai, Liftschacht Gleis 6
Passage Sihlquai, Liftschacht Gleis 7
Passage, Zugang Pe

In [116]:
from typing import Dict, NamedTuple

NrRange = namedtuple("NrRange", ["low", "high"], defaults=[None])
StationMapping = Dict[NrRange, str]
CantonMapping = Dict[str, str]


def map_stations(
    canton: str = None, cantons: CantonMapping = None, nr_ranges: StationMapping = None
) -> None:
    """
    Map spr_data records to Intervista Station names in-place, either per canton by 
    Flaechennummer ranges NrRange(low, high), or by directly mapping from a canton 
    to one station
    """
    if nr_ranges is not None and canton is not None:
        for nr_range, station_name in nr_ranges.items():
            if nr_range.high is None:
                nr_range = NrRange(nr_range.low, nr_range.low)
            spr_data.loc[
                (spr_data["Gebiet Code"] == canton)
                & (spr_data["Nummer"].isin(range(nr_range.low, nr_range.high + 1))),
                "Station",
            ] = station_name
    if cantons is not None:
        for canton, station_name in cantons.items():
            spr_data.loc[spr_data["Gebiet Code"] == canton, "Station"] = station_name

### ZH

In [117]:
collect(spr_data.loc[spr_data["Gebiet Code"] == "ZH", "Nummer"], sep=" | ")

'680759.0 | 680760.0 | 680761.0 | 680763.0 | 680764.0 | 680765.0 | 680766.0 | 680767.0 | 680768.0 | 680769.0 | 680770.0 | 680776.0 | 680777.0 | 680814.0 | 680815.0 | 680816.0 | 680824.0 | 680827.0 | 680828.0 | 680829.0 | 680831.0 | 680846.0 | 680847.0 | 680848.0 | 680858.0 | 680873.0 | 680874.0 | 680875.0 | 680876.0 | 688299.0 | 688303.0 | 689938.0 | 689940.0 | 689942.0 | 689944.0 | 689946.0 | 689948.0 | 689950.0 | 690717.0 | 690723.0 | 690910.0 | 690914.0 | 690918.0 | 690922.0 | 705698.0 | 705699.0 | 705700.0 | 705701.0 | 705702.0 | 705703.0 | 705704.0 | 705705.0'

In [118]:
map_stations(
    canton="ZH",
    nr_ranges={
        NrRange(680759, 680831): "Zürich HB",
        NrRange(680846, 680848): "Zürich Stadelhofen",
        NrRange(680858, 680858): "Zürich Enge",
        NrRange(680873, 680876): "Winterthur",
        NrRange(688299, 688303): "Zürich Flughafen",
        NrRange(689938, 690717): "Zürich HB",  # Sihlquai-Bahnhof, Gleis-Unterführungen
        NrRange(690723): "Winterthur",
        NrRange(690910, 690922): "Zürich HB",  # Bahnhofplatz
        NrRange(705698, 705701): "Winterthur",
        NrRange(705702, 705703): "Zürich Stadelhofen",
        NrRange(705704, 705705): "Zürich Oerlikon",
    }
)

In [119]:
spr_data.query("Station == 'Winterthur'")

,index,WT,Flaeche_ID,Anbieter,Spr_Flaeche_ID,Time,Count,Nummer,PF,Position-Nr.,Gebiet,Gebietsbez.,Gebiet Code,Adresszusatz,STM,EN-Nr.,NG-Nr.,Station
483,483,Dienstag,680873,1,1680873,0,71.130863,680873.0,Rail eBoard,DA01,23001.0,Winterthur-Zentrum,ZH,"Gleis 4, Sektor B, vis-à-vis Hausperron","Bahnhof, WaitingZone",416730.0,212784.0,Winterthur
484,484,Donnerstag,680873,1,1680873,0,95.704505,680873.0,Rail eBoard,DA01,23001.0,Winterthur-Zentrum,ZH,"Gleis 4, Sektor B, vis-à-vis Hausperron","Bahnhof, WaitingZone",416730.0,212784.0,Winterthur
485,485,Freitag,680873,1,1680873,0,116.623583,680873.0,Rail eBoard,DA01,23001.0,Winterthur-Zentrum,ZH,"Gleis 4, Sektor B, vis-à-vis Hausperron","Bahnhof, WaitingZone",416730.0,212784.0,Winterthur
486,486,Mittwoch,680873,1,1680873,0,82.528785,680873.0,Rail eBoard,DA01,23001.0,Winterthur-Zentrum,ZH,"Gleis 4, Sektor B, vis-à-vis Hausperron","Bahnhof, WaitingZone",416730.0,212784.0,Winterthur
487,487,Montag,680873,1,1680873,0,83.532569,680873.0,Rail eBoard,DA01,23001.0,Winterthur-Zentrum,ZH,"Gleis 4, Sektor B, vis-à-vis Hausperron","Bahnhof, WaitingZone",416730.0,212784.0,Winterthur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359368,359368,Freitag,705701,1,1705701,85500000000,102.974331,705701.0,Rail ePanel,DD01,23001.0,Winterthur-Zentrum,ZH,Perronpassage,Bahnhof,416998.0,212821.0,Winterthur
359369,359369,Mittwoch,705701,1,1705701,85500000000,44.345717,705701.0,Rail ePanel,DD01,23001.0,Winterthur-Zentrum,ZH,Perronpassage,Bahnhof,416998.0,212821.0,Winterthur
359370,359370,Montag,705701,1,1705701,85500000000,35.149297,705701.0,Rail ePanel,DD01,23001.0,Winterthur-Zentrum,ZH,Perronpassage,Bahnhof,416998.0,212821.0,Winterthur
359371,359371,Samstag,705701,1,1705701,85500000000,137.065866,705701.0,Rail ePanel,DD01,23001.0,Winterthur-Zentrum,ZH,Perronpassage,Bahnhof,416998.0,212821.0,Winterthur


### GE

In [120]:
collect(spr_data.loc[spr_data["Gebiet Code"] == "GE", "Nummer"], sep=" | ")

'680802.0 | 680803.0 | 680804.0 | 680805.0 | 680806.0 | 680807.0 | 680903.0 | 680904.0 | 690726.0'

In [121]:
map_stations(
    canton="GE",
    nr_ranges={
        NrRange(680802, 690726): "Genève Cornavin",
    }
)

In [122]:
spr_data.query("Station == 'Genève Cornavin'")

,index,WT,Flaeche_ID,Anbieter,Spr_Flaeche_ID,Time,Count,Nummer,PF,Position-Nr.,Gebiet,Gebietsbez.,Gebiet Code,Adresszusatz,STM,EN-Nr.,NG-Nr.,Station
329,329,Dienstag,680802,1,1680802,0,12.132453,680802.0,Rail ePanel,1DA01,662101.0,Genève-Centre,GE,"Passage Aile Ouest, voies 1 - 2, kiosque, shop...",Bahnhof,417117.0,212913.0,Genève Cornavin
330,330,Donnerstag,680802,1,1680802,0,15.912037,680802.0,Rail ePanel,1DA01,662101.0,Genève-Centre,GE,"Passage Aile Ouest, voies 1 - 2, kiosque, shop...",Bahnhof,417117.0,212913.0,Genève Cornavin
331,331,Freitag,680802,1,1680802,0,17.280033,680802.0,Rail ePanel,1DA01,662101.0,Genève-Centre,GE,"Passage Aile Ouest, voies 1 - 2, kiosque, shop...",Bahnhof,417117.0,212913.0,Genève Cornavin
332,332,Mittwoch,680802,1,1680802,0,13.064491,680802.0,Rail ePanel,1DA01,662101.0,Genève-Centre,GE,"Passage Aile Ouest, voies 1 - 2, kiosque, shop...",Bahnhof,417117.0,212913.0,Genève Cornavin
333,333,Montag,680802,1,1680802,0,14.649434,680802.0,Rail ePanel,1DA01,662101.0,Genève-Centre,GE,"Passage Aile Ouest, voies 1 - 2, kiosque, shop...",Bahnhof,417117.0,212913.0,Genève Cornavin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359144,359144,Freitag,690726,1,1690726,85500000000,141.713554,690726.0,Rail eBoard,1DF01,662101.0,Genève-Centre,GE,"Hall de la gare, accès Passage Aile Est , shop...",Bahnhof,416698.0,212754.0,Genève Cornavin
359145,359145,Mittwoch,690726,1,1690726,85500000000,70.549837,690726.0,Rail eBoard,1DF01,662101.0,Genève-Centre,GE,"Hall de la gare, accès Passage Aile Est , shop...",Bahnhof,416698.0,212754.0,Genève Cornavin
359146,359146,Montag,690726,1,1690726,85500000000,61.345608,690726.0,Rail eBoard,1DF01,662101.0,Genève-Centre,GE,"Hall de la gare, accès Passage Aile Est , shop...",Bahnhof,416698.0,212754.0,Genève Cornavin
359147,359147,Samstag,690726,1,1690726,85500000000,159.441696,690726.0,Rail eBoard,1DF01,662101.0,Genève-Centre,GE,"Hall de la gare, accès Passage Aile Est , shop...",Bahnhof,416698.0,212754.0,Genève Cornavin


### VD

In [123]:
collect(spr_data.loc[spr_data["Gebiet Code"] == "VD", "Nummer"], sep=" | ")

'680797.0 | 680798.0 | 680800.0 | 680801.0 | 680897.0 | 680898.0 | 711290.0 | 711291.0 | 711292.0 | 711293.0 | 711296.0 | 711297.0 | 711298.0 | 711299.0 | 711300.0 | 711301.0 | 711302.0 | 711303.0 | 711304.0 | 711305.0 | 711306.0 | 711307.0 | 711308.0 | 711309.0 | 711310.0 | 711311.0 | 711312.0 | 711313.0 | 711314.0'

In [124]:
map_stations(
    canton="VD",
    nr_ranges={
        NrRange(680797, 711299): "Lausanne",
        NrRange(711300, 711308): "M2", # M2 Gare Lausanne
        NrRange(711309, 711314): "M2", # M2 Grancy
    }
)

In [125]:
spr_data.query("Station == 'M2'")

,index,WT,Flaeche_ID,Anbieter,Spr_Flaeche_ID,Time,Count,Nummer,PF,Position-Nr.,Gebiet,Gebietsbez.,Gebiet Code,Adresszusatz,STM,EN-Nr.,NG-Nr.,Station
2688,2688,Dienstag,711300,1,1711300,0,11.949766,711300.0,Rail ePanel,1DA01,558601.0,Lausanne-Centre,VD,"Place de la Gare, entrée Quai direction Ouchy",Bahnhof,417100.0,212899.0,M2
2689,2689,Donnerstag,711300,1,1711300,0,15.984866,711300.0,Rail ePanel,1DA01,558601.0,Lausanne-Centre,VD,"Place de la Gare, entrée Quai direction Ouchy",Bahnhof,417100.0,212899.0,M2
2690,2690,Freitag,711300,1,1711300,0,18.373168,711300.0,Rail ePanel,1DA01,558601.0,Lausanne-Centre,VD,"Place de la Gare, entrée Quai direction Ouchy",Bahnhof,417100.0,212899.0,M2
2691,2691,Mittwoch,711300,1,1711300,0,13.067901,711300.0,Rail ePanel,1DA01,558601.0,Lausanne-Centre,VD,"Place de la Gare, entrée Quai direction Ouchy",Bahnhof,417100.0,212899.0,M2
2692,2692,Montag,711300,1,1711300,0,13.116964,711300.0,Rail ePanel,1DA01,558601.0,Lausanne-Centre,VD,"Place de la Gare, entrée Quai direction Ouchy",Bahnhof,417100.0,212899.0,M2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361223,361223,Freitag,711314,1,1711314,85500000000,3.446263,711314.0,Rail ePanel,1DE03,558602.0,Lausanne-Ouchy-Vidy,VD,"Quai direction Croisettes, porte palière",Bahnhof,417104.0,212903.0,M2
361224,361224,Mittwoch,711314,1,1711314,85500000000,1.452324,711314.0,Rail ePanel,1DE03,558602.0,Lausanne-Ouchy-Vidy,VD,"Quai direction Croisettes, porte palière",Bahnhof,417104.0,212903.0,M2
361225,361225,Montag,711314,1,1711314,85500000000,1.064790,711314.0,Rail ePanel,1DE03,558602.0,Lausanne-Ouchy-Vidy,VD,"Quai direction Croisettes, porte palière",Bahnhof,417104.0,212903.0,M2
361226,361226,Samstag,711314,1,1711314,85500000000,4.532524,711314.0,Rail ePanel,1DE03,558602.0,Lausanne-Ouchy-Vidy,VD,"Quai direction Croisettes, porte palière",Bahnhof,417104.0,212903.0,M2


### BE

In [126]:
collect(spr_data.loc[spr_data["Gebiet Code"] == "BE", "Nummer"], sep=" | ")

'680778.0 | 680779.0 | 680781.0 | 680782.0 | 680783.0 | 680861.0 | 680862.0 | 690699.0 | 691900.0 | 691903.0 | 711340.0 | 711341.0 | 721646.0 | 721648.0 | 721650.0 | 721652.0 | 721654.0 | 721656.0 | 721658.0 | 721660.0 | 721662.0 | 721664.0 | 721666.0 | 721668.0 | 721670.0 | 721672.0 | 721674.0 | 721676.0 | 721678.0 | 721680.0 | 721682.0 | 721684.0 | 721686.0 | 721688.0 | 721690.0 | 721692.0 | 721694.0 | 721696.0 | 721698.0 | 721700.0'

In [127]:
map_stations(
    canton="BE",
    nr_ranges={
        NrRange(680778, 680862): "Bern",
        NrRange(690699): "Biel/Bienne",
        NrRange(691900, 691903): "Bern",
        NrRange(711340, 711341): "Biel/Bienne",
        NrRange(721646, 721700): "Bern",
    }
)

In [128]:
spr_data.query("Station == 'Biel/Bienne'")

,index,WT,Flaeche_ID,Anbieter,Spr_Flaeche_ID,Time,Count,Nummer,PF,Position-Nr.,Gebiet,Gebietsbez.,Gebiet Code,Adresszusatz,STM,EN-Nr.,NG-Nr.,Station
651,651,Dienstag,690699,1,1690699,0,15.243410,690699.0,Rail eBoard,DH01,37101.0,Biel-Zentrum,BE,Bahnhofshalle,Bahnhof,416724.0,212778.0,Biel/Bienne
652,652,Donnerstag,690699,1,1690699,0,20.524197,690699.0,Rail eBoard,DH01,37101.0,Biel-Zentrum,BE,Bahnhofshalle,Bahnhof,416724.0,212778.0,Biel/Bienne
653,653,Freitag,690699,1,1690699,0,24.829137,690699.0,Rail eBoard,DH01,37101.0,Biel-Zentrum,BE,Bahnhofshalle,Bahnhof,416724.0,212778.0,Biel/Bienne
654,654,Mittwoch,690699,1,1690699,0,17.855035,690699.0,Rail eBoard,DH01,37101.0,Biel-Zentrum,BE,Bahnhofshalle,Bahnhof,416724.0,212778.0,Biel/Bienne
655,655,Montag,690699,1,1690699,0,17.939523,690699.0,Rail eBoard,DH01,37101.0,Biel-Zentrum,BE,Bahnhofshalle,Bahnhof,416724.0,212778.0,Biel/Bienne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361237,361237,Freitag,711341,1,1711341,85500000000,71.612537,711341.0,Rail ePanel,DD03,37101.0,Biel-Zentrum,BE,"Halle, Hauptausgang Bahnhofplatz",Bahnhof,417105.0,212904.0,Biel/Bienne
361238,361238,Mittwoch,711341,1,1711341,85500000000,30.590844,711341.0,Rail ePanel,DD03,37101.0,Biel-Zentrum,BE,"Halle, Hauptausgang Bahnhofplatz",Bahnhof,417105.0,212904.0,Biel/Bienne
361239,361239,Montag,711341,1,1711341,85500000000,24.447216,711341.0,Rail ePanel,DD03,37101.0,Biel-Zentrum,BE,"Halle, Hauptausgang Bahnhofplatz",Bahnhof,417105.0,212904.0,Biel/Bienne
361240,361240,Samstag,711341,1,1711341,85500000000,96.188792,711341.0,Rail ePanel,DD03,37101.0,Biel-Zentrum,BE,"Halle, Hauptausgang Bahnhofplatz",Bahnhof,417105.0,212904.0,Biel/Bienne


### TI

In [129]:
collect(spr_data.loc[spr_data["Gebiet Code"] == "TI", "Nummer"], sep=" | ")

'711180.0 | 713103.0 | 713104.0 | 713105.0 | 713106.0'

In [130]:
map_stations(
    canton="TI",
    nr_ranges={
        NrRange(711180): "Bellinzona",
        NrRange(713103, 713106): "Lugano", 
    }
)

In [131]:
spr_data.query("Station == 'Bellinzona'")

,index,WT,Flaeche_ID,Anbieter,Spr_Flaeche_ID,Time,Count,Nummer,PF,Position-Nr.,Gebiet,Gebietsbez.,Gebiet Code,Adresszusatz,STM,EN-Nr.,NG-Nr.,Station
2548,2548,Dienstag,711180,1,1711180,0,5.369892,711180.0,Rail eBoard,DA01,500201.0,Bellinzona-Centro,TI,atrio,"Bahnhof, WaitingZone",417112.0,212909.0,Bellinzona
2549,2549,Donnerstag,711180,1,1711180,0,7.219979,711180.0,Rail eBoard,DA01,500201.0,Bellinzona-Centro,TI,atrio,"Bahnhof, WaitingZone",417112.0,212909.0,Bellinzona
2550,2550,Freitag,711180,1,1711180,0,8.865614,711180.0,Rail eBoard,DA01,500201.0,Bellinzona-Centro,TI,atrio,"Bahnhof, WaitingZone",417112.0,212909.0,Bellinzona
2551,2551,Mittwoch,711180,1,1711180,0,6.297286,711180.0,Rail eBoard,DA01,500201.0,Bellinzona-Centro,TI,atrio,"Bahnhof, WaitingZone",417112.0,212909.0,Bellinzona
2552,2552,Montag,711180,1,1711180,0,6.397715,711180.0,Rail eBoard,DA01,500201.0,Bellinzona-Centro,TI,atrio,"Bahnhof, WaitingZone",417112.0,212909.0,Bellinzona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360985,360985,Freitag,711180,1,1711180,85500000000,21.432006,711180.0,Rail eBoard,DA01,500201.0,Bellinzona-Centro,TI,atrio,"Bahnhof, WaitingZone",417112.0,212909.0,Bellinzona
360986,360986,Mittwoch,711180,1,1711180,85500000000,9.215500,711180.0,Rail eBoard,DA01,500201.0,Bellinzona-Centro,TI,atrio,"Bahnhof, WaitingZone",417112.0,212909.0,Bellinzona
360987,360987,Montag,711180,1,1711180,85500000000,7.349300,711180.0,Rail eBoard,DA01,500201.0,Bellinzona-Centro,TI,atrio,"Bahnhof, WaitingZone",417112.0,212909.0,Bellinzona
360988,360988,Samstag,711180,1,1711180,85500000000,28.845034,711180.0,Rail eBoard,DA01,500201.0,Bellinzona-Centro,TI,atrio,"Bahnhof, WaitingZone",417112.0,212909.0,Bellinzona


## Simple cases (one Station per canton)

In [132]:
map_stations(
    cantons={
        "AG": "Aarau",
        "BS": "Basel SBB",
        "FR": "Fribourg",
        "GR": "Chur",
        "LU": "Luzern",
        "NE": "Neuchatel",
        "SG": "St. Gallen",
        "SO": "Olten",
        "VS": "Brig",
        "ZG": "Zug",
    }
)

In [133]:
spr_data.query("Station == 'Olten'")

,index,WT,Flaeche_ID,Anbieter,Spr_Flaeche_ID,Time,Count,Nummer,PF,Position-Nr.,Gebiet,Gebietsbez.,Gebiet Code,Adresszusatz,STM,EN-Nr.,NG-Nr.,Station
679,679,Dienstag,690714,1,1690714,0,26.677106,690714.0,Rail eBoard,DH01,258102.0,Olten-Bifang-Hardfeld,SO,Bahnhofshalle,Bahnhof,416722.0,212776.0,Olten
680,680,Donnerstag,690714,1,1690714,0,35.942928,690714.0,Rail eBoard,DH01,258102.0,Olten-Bifang-Hardfeld,SO,Bahnhofshalle,Bahnhof,416722.0,212776.0,Olten
681,681,Freitag,690714,1,1690714,0,43.689736,690714.0,Rail eBoard,DH01,258102.0,Olten-Bifang-Hardfeld,SO,Bahnhofshalle,Bahnhof,416722.0,212776.0,Olten
682,682,Mittwoch,690714,1,1690714,0,30.861689,690714.0,Rail eBoard,DH01,258102.0,Olten-Bifang-Hardfeld,SO,Bahnhofshalle,Bahnhof,416722.0,212776.0,Olten
683,683,Montag,690714,1,1690714,0,31.253190,690714.0,Rail eBoard,DH01,258102.0,Olten-Bifang-Hardfeld,SO,Bahnhofshalle,Bahnhof,416722.0,212776.0,Olten
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359116,359116,Freitag,690714,1,1690714,85500000000,106.614486,690714.0,Rail eBoard,DH01,258102.0,Olten-Bifang-Hardfeld,SO,Bahnhofshalle,Bahnhof,416722.0,212776.0,Olten
359117,359117,Mittwoch,690714,1,1690714,85500000000,46.156385,690714.0,Rail eBoard,DH01,258102.0,Olten-Bifang-Hardfeld,SO,Bahnhofshalle,Bahnhof,416722.0,212776.0,Olten
359118,359118,Montag,690714,1,1690714,85500000000,36.276550,690714.0,Rail eBoard,DH01,258102.0,Olten-Bifang-Hardfeld,SO,Bahnhofshalle,Bahnhof,416722.0,212776.0,Olten
359119,359119,Samstag,690714,1,1690714,85500000000,143.133870,690714.0,Rail eBoard,DH01,258102.0,Olten-Bifang-Hardfeld,SO,Bahnhofshalle,Bahnhof,416722.0,212776.0,Olten


## Matched all of them?

In [134]:
with value(spr_data.Station.value_counts(dropna=False)) as records:
    display(pd.DataFrame(dict(Records=records, Displays=records // 672)))

,Records,Displays
Bern,24864,37
Zürich HB,22176,33
M2,10080,15
Lausanne,9408,14
Luzern,6720,10
Winterthur,6048,9
Genève Cornavin,6048,9
Basel SBB,6048,9
Chur,4704,7
Zürich Stadelhofen,3360,5


# Store finished data

In [135]:
with project_dir("axinova"):
    store_bin(spr_data, "spr_data_mapped.feather")

17:11:03 [INFO] Started storing feather file ...
17:11:03 [INFO] Writing to file C:\Users\kpf\data\axinova\spr_data_mapped.feather
17:11:03 [INFO] Written 11.2 MB
17:11:03 [INFO] ... finished storing feather file in 0.02s (0.03s CPU)
